In [1]:
import tweepy
consumer_key = "VqcxzBOXycA5eSY4DVh8gdbcN"
consumer_secret = "kLkViKhCJ9VaXAeuvNr2bEBgC5pcYDOi5lwh5vXuTtc0bpL9L7"
access_token = "400927565-O9z7W7ucgL7esDLkRg2e4JSTfiNSTixqlf2tmC7i"
access_token_secret = "Oe2ohU0flKGf1HUn0ZkVTXmJ2tA3IgXCjkyFgo9x75imF"

In [2]:
#creating the authentication object
auth= tweepy.OAuthHandler(consumer_key, consumer_secret)
#setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
#creating the API object while passing in auth information
api = tweepy.API(auth)

In [29]:
tweets = api.search("qatarairways OR qrsupport", lang='en',rpp=100)

In [30]:
# foreach through all tweets pulled
for tweet in tweets:
   # printing the text stored inside the tweet object
   print(tweet.user.screen_name,"Tweeted:",tweet.text)

reem101131 Tweeted: RT @qatarairways: Game on #Oman! We are the Official Airline Partner for three of Oman’s Ministry of Education sports programmes and the Om…
alpamaika Tweeted: @qatarairways 24 h from Bangkok to Rome is extremely long, arrived in Doha 40 min before the connection flight and… https://t.co/aaOOsXR5Rm
medshare Tweeted: RT @AirlinkFlight: @qatarairways is making a huge difference for #CycloneIdai survivors. They've donated a flight carrying @SavetheChildren…
AliHasa_n Tweeted: #Flights on time 2017

#Qantas 86.2%
#JapanAirlines 85.3%
#Singapore 84.1%
#AllNippon 83.8%
#QatarAirways 82.9%… https://t.co/nVbWR1git4
qrsupport Tweeted: @WrisJarrett Investigate further and revert back to you. Please send to tell-us@qatarairways.com.qa
Thank you. 2/2.
GulfTimes_QATAR Tweeted: #US govt should not restrict #QatarAirways from #flying to the country, say 3 #airlines
#aviation… https://t.co/cDRwq5BNUT
qrsupport Tweeted: @WrisJarrett We can't be apologetic enough for the inconvenienc

In [32]:
import pandas as pd


In [36]:
with open(tweets.text) as infile, open('tweets.csv','w') as outfile: 
    for line in infile: 
        outfile.write(line.replace(' ',','))

AttributeError: 'SearchResults' object has no attribute 'text'